Scratch

In [1]:
test = [
            dict( n=6, val= 5, moe=1, newBlockP1=0.8, newBlockP2=0.2),
            dict( n=10, val= 15, moe=2, newBlockP1=0.3, newBlockP2=0.7)
        ]




In [2]:
## number of people
import pandas

df= pandas.DataFrame(test)
df

,moe,n,newBlockP1,newBlockP2,val
0,1,6,0.8,0.2,5
1,2,10,0.3,0.7,15


In [16]:
## number of people

## deterministic

df[['newBlockP1','newBlockP2']]

,newBlockP1,newBlockP2
0,0.8,0.2
1,0.3,0.7


In [3]:
nam=list(df.columns.values)
nam

['moe', 'n', 'newBlockP1', 'newBlockP2', 'val']

In [4]:
nam.remove("moe")
nam.remove('n')
nam.remove('val')
nam

['newBlockP1', 'newBlockP2']

In [89]:

for i in range(df.shape[0]):
    print(df['n'][i]*df[i:(i+1)][nam])
   # how do I store these, then want to sum over columns 

   newBlockP1  newBlockP2
0         4.8         1.2
   newBlockP1  newBlockP2
1         3.0         7.0


In [9]:
## with moe
import numpy
for i in range(df.shape[0]):
    se = df['n'][i] / 1.645 ## convert moe to se
    new_n = round(numpy.random.normal(df['n'][i], se))
    ## use moe to introduce randomness into number in bin
    new_n = int(new_n) ## clean it up
    print(new_n*df[i:(i+1)][nam])

   # how do I store these, then want to sum over columns 

   newBlockP1  newBlockP2
0         2.4         0.6
   newBlockP1  newBlockP2
1         4.5        10.5


In [7]:
## mean
## go through by columns
## deterministic

for i in nam:
    print(sum(df['n']*df[i]*df['val'])/sum(df['n']*df[i]))

8.846153846153845
13.536585365853659


In [13]:
## mean
## go through by columns
## random

sim_n = []
for i in range(df.shape[0]):
    se = df['n'][i] / 1.645 ## convert moe to se
    new_n = round(numpy.random.normal(df['n'][i], se))
    ## use moe to introduce randomness into number in bin
    new_n = int(new_n) ## clean it up
    sim_n.append(new_n)

for i in nam:
    print(sum(new_n*df[i]*df['val'])/sum(new_n*df[i]))

7.7272727272727275
12.777777777777779


In [ ]:
## median
## deterministic



In [24]:
import math
household_income_2013_acs5 = [
            dict(min=math.nan, max=9999, n=6, moe=1),
            dict(min=10000, max=14999, n=1, moe=1),
            dict(min=15000, max=19999, n=8, moe=1),
            dict(min=20000, max=24999, n=7, moe=1),
            dict(min=25000, max=29999, n=2, moe=1),
            dict(min=30000, max=34999, n=900, moe=8),
            dict(min=35000, max=39999, n=7, moe=1),
            dict(min=40000, max=44999, n=4, moe=1),
            dict(min=45000, max=49999, n=8, moe=1),
            dict(min=50000, max=59999, n=6, moe=1),
            dict(min=60000, max=74999, n=7, moe=1),
            dict(min=75000, max=99999, n=2, moe=0.25),
            dict(min=100000, max=124999, n=7, moe=1),
            dict(min=125000, max=149999, n=10, moe=1),
            dict(min=150000, max=199999, n=8, moe=1),
            dict(min=200000, max=math.nan, n=18, moe=10)
        ]

household_income_la_2013_acs1 = [
      dict(min=0, max=9999, n=1382),
      dict(min=10000, max=14999, n=2377),
      dict(min=15000, max=19999, n=1332),
      dict(min=20000, max=24999, n=3129),
      dict(min=25000, max=29999, n=1927),
      dict(min=30000, max=34999, n=1825),
      dict(min=35000, max=39999, n=1567),
      dict(min=40000, max=44999, n=1996),
      dict(min=45000, max=49999, n=1757),
      dict(min=50000, max=59999, n=3523),
      dict(min=60000, max=74999, n=4360),
      dict(min=75000, max=99999, n=6424),
      dict(min=100000, max=124999, n=5257),
      dict(min=125000, max=149999, n=3485),
      dict(min=150000, max=199999, n=2926),
      dict(min=200000, max=500000, n=4215)
  ]


test_input = [household_income_2013_acs5,household_income_la_2013_acs1]

makeup_1 = [0.8, 0.2]
makeup_2 =  [0.3, 0.7]

makeup_input = [makeup_1, makeup_2]


2377

In [147]:
def disaggregate_sum(data_list, makeup, deterministic = True, simulations=50):

    if deterministic:
        results = []
        grand_total = []
        for i in range(len(data_list)):
            total = []
            for j in range(0, len(data_list[i])):
                total.append(data_list[i][j]['n'])
            grand_total.append(sum(total))
            results.append([grand_total[i]*z for z in makeup[i]]) ## store this row
        rr = numpy.vstack(results)
        est = numpy.apply_along_axis(sum, 0, rr) 
        return est, None
    else:
        simulation_results = []
        for s in range(simulations):
            results = []
            grand_total = []
            for i in range(len(data_list)):
                total = []
                for j in range(len(data_list[i])):
                    se = data_list[i][j]['n'] / 1.645 ## convert moe to se
                    new_n = round(numpy.random.normal(data_list[i][j]['n'], se))
                    ## use moe to introduce randomness into number in bin
                    new_n = int(new_n) ## clean it up
                    total.append(new_n)
                grand_total.append(sum(total))
                results.append([grand_total[i]*z for z in makeup[i]]) ## store this row
            rr = numpy.vstack(results)
            simulation_results.append(numpy.apply_along_axis(sum, 0, rr)) 
        ss = numpy.vstack(simulation_results)
        est = numpy.apply_along_axis(numpy.mean, 0, ss)
        t1 = numpy.apply_along_axis(numpy.quantile, axis=0, arr =ss, q=0.95) - est
        t2 = est - numpy.apply_along_axis(numpy.quantile, axis=0, arr = ss, q=0.05)
        margin_of_error = numpy.amax(numpy.column_stack((t1,t2)), 1)
        return est, margin_of_error

In [88]:
disaggregate_sum(test_input, makeup_input, deterministic = True, simulations=50)

(array([15045.4, 33437.6]), None)

In [148]:
disaggregate_sum(test_input, makeup_input, deterministic = False, simulations=50)

(array([14950.984, 33270.996]), array([4125.539, 9169.806]))